<a href="https://colab.research.google.com/github/SriRamK345/Movie-Sentiment-Analysis/blob/main/Movie_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp "/content/drive/MyDrive/kaggle.json" ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download nltkdata/movie-review

In [ ]:
! unzip /content/movie-review.zip

In [ ]:
import pandas as pd
import numpy as np
import warnings
# Settings the warnings to be ignored
warnings.filterwarnings("ignore")
import re

In [ ]:
df = pd.read_csv("/content/movie_review.csv")
df.head()

In [ ]:
df.tag.unique()

## Data Preprocessing

In [ ]:
# Contraction Map
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}


In [ ]:
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):

    contractions_pattern = re.compile("({})".format("|".join(contraction_mapping.keys())),
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub(" ' ", "", expanded_text)
    return expanded_text

df["text_comp"] = df["text"].apply(expand_contractions)

In [ ]:
df["text_comp"]

### Removing Special Characters

In [ ]:
import string
import nltk # this needs to be explictly imported
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download("stopwords")
nltk.download("wordnet")

In [ ]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()
number_pattern = re.compile(r"\d+")

def preprocess_test(text):
  text = number_pattern.sub("", text)  # Remove numbers
  text = text.translate(str.maketrans("", "", string.punctuation)).lower()  # Remove punctuation and lowercase
  text = [lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words] # Lemmatization & stop word removal
  return " ".join(text)

df["text_pro"] = df["text"].apply(preprocess_test)

In [ ]:
df["text_pro"]

In [ ]:
X = df["text_pro"]
y = df["tag"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
# Pipeline
pipe = Pipeline([("tfidf", TfidfVectorizer()), ("cNB", MultinomialNB())])

In [ ]:
# Visualize Pipeline
from sklearn import set_config
set_config(display="diagram")

In [ ]:
pipe

In [ ]:
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

In [ ]:
df.tag.value_counts()

In [ ]:
# Evaluation

print(classification_report(y_test, y_pred))

In [ ]:
# confusion
confusion_matrix(y_test, y_pred)

In [ ]:
# visulization
import seaborn as sns
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt="d")

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

words_list = df[df["tag"]=="pos"]["text"].unique().tolist()
positive_words = " ".join(words_list)

pos_wordcloud =  WordCloud(
                  width=500, height = 300,
                  stopwords=stop_words).generate(positive_words)

plt.figure(figsize=(8, 8), facecolor = None)
plt.imshow(pos_wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.title("Positive Words")
plt.show()

In [ ]:
words_list = df[df["tag"]=="neg"]["text"].unique().tolist()
negative_words = " ".join(words_list)

neg_wordcloud =  WordCloud(
                  width=500, height = 300,
                  stopwords=stop_words).generate(negative_words)

plt.figure(figsize=(8, 8), facecolor = None)
plt.imshow(neg_wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.title("Negative Words")
plt.show()

In [ ]:
sample1 = "I loved this movie"
pipe.predict([sample1])

In [ ]:
sample2 = "movie was boring"
pipe.predict([sample2])

In [ ]:
list_sam = ["I do not love this movie", "movie was boring", "movie is a rotten tomato","entertaining well"]
pipe.predict(list_sam)

In [ ]:
df[["text","text_comp","text_pro"]].head()

In [ ]:
# Install spacy and download the English language model (if not already done):
# !pip install spacy==3.6.1
!python -m spacy download en_core_web_sm

In [ ]:
import spacy
from spacy import displacy
# Load spaCy's English language model:
nlp = spacy.load("en_core_web_sm")

In [ ]:
def preprocess_text_spacy(text):
    #text = expand_contractions(text)
    doc = nlp(text)

    # Create a list of stop words from spaCy's vocabulary:
    stop_words = nlp.Defaults.stop_words

    # Lemmatization, stop word removal, punctuation removal, and lowercasing:
    text = [token.lemma_.lower() for token in doc
            if token.text.lower() not in stop_words
            and not token.is_punct
            and token.text not in string.punctuation]

    # Remove numbers:
    number_pattern = re.compile(r"\d+")
    text = [word for word in text if not number_pattern.match(word)]

    return " ".join(text)

# Apply the preprocessing function to your text data:
df["text_pro_spacy"] = df["text"].apply(preprocess_text_spacy)

In [ ]:
df

In [ ]:
def preprocess_text(text):
    """
    Preprocesses a single text by performing the following steps:
    1. Lowercasing
    2. Removing special characters and numbers
    3. Tokenization
    4. Removing stop words
    5. Lemmatization
    6. Removing short words (less than 3 characters)

    Args:
        text (str): The input text.

    Returns:
        str: The preprocessed text.
    """
    # Convert to lowercase
    text = text.lower()

    # Remove special characters and numbers
    text = re.sub(r"[^a-zA-Z\s]", "", text)

    # Process text using spaCy
    doc = nlp(text)

    # Tokenization, stop word removal, and lemmatization in one step
    tokens = [token.lemma_ for token in doc if not token.is_stop]

    # Join tokens back into a single string
    return " ".join(tokens)

In [ ]:
df["text_pro_spacy"] = df["text"].apply(preprocess_text)

In [ ]:
df[["text","text_pro","text_pro_spacy"]].tail()